In [2]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
import scipy.cluster.hierarchy as hcluster

In [3]:
# topics = ["ml1", "ml2", "ml3", "ml4", "ml5", "ml6", "ml7", "ml8", "ml9", "ml10", "ml11", "ml12", "ml13", "ml14", "ml15", "ml16", "ml17", "ml18", "ml19"]
# print "The titles are ", topics

In [4]:
import glob
import numpy as np

filePath = './books'
fileCounter = len(glob.glob(filePath + "*.txt"))

paragraphs = []
topics = []
book_sentence_pairs=[]
for index, filename in enumerate(glob.glob(filePath + '/*.txt')):
    print(filename)
    file = open(filename, 'r')
    txt = file.read()
    txt = txt.replace('\n', '').split('.')
    txt = filter(None, txt)
    print len(txt)
    for i in np.arange(0, len(txt)):
        topics.append('book: %d sentence: %d' % (index, i))
        book_sentence_pairs.append([index,i])
        paragraphs.append(txt[i])

print len(topics)
print len(paragraphs)
print book_sentence_pairs

./books/d3.txt
96
./books/d1.txt
56
./books/d2.txt
21
173
173
[[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [0, 10], [0, 11], [0, 12], [0, 13], [0, 14], [0, 15], [0, 16], [0, 17], [0, 18], [0, 19], [0, 20], [0, 21], [0, 22], [0, 23], [0, 24], [0, 25], [0, 26], [0, 27], [0, 28], [0, 29], [0, 30], [0, 31], [0, 32], [0, 33], [0, 34], [0, 35], [0, 36], [0, 37], [0, 38], [0, 39], [0, 40], [0, 41], [0, 42], [0, 43], [0, 44], [0, 45], [0, 46], [0, 47], [0, 48], [0, 49], [0, 50], [0, 51], [0, 52], [0, 53], [0, 54], [0, 55], [0, 56], [0, 57], [0, 58], [0, 59], [0, 60], [0, 61], [0, 62], [0, 63], [0, 64], [0, 65], [0, 66], [0, 67], [0, 68], [0, 69], [0, 70], [0, 71], [0, 72], [0, 73], [0, 74], [0, 75], [0, 76], [0, 77], [0, 78], [0, 79], [0, 80], [0, 81], [0, 82], [0, 83], [0, 84], [0, 85], [0, 86], [0, 87], [0, 88], [0, 89], [0, 90], [0, 91], [0, 92], [0, 93], [0, 94], [0, 95], [1, 0], [1, 1], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [1, 9], [1,

In [5]:
stopwords = nltk.corpus.stopwords.words('english')
print stopwords

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u

In [6]:
# Now we want to break a word into its root using a stemmer.
#We use the snowball.
#snowball is better than porter stemmer

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [7]:
#First the parageraph is tokenized by sentence then by word
def tokenize(paragraph) :
    tokenList = [word for sentence in nltk.sent_tokenize(paragraph) for word in nltk.word_tokenize(sentence)]
    
    filteredTokens = []
    
    for token in tokenList:
        if re.search('[a-zA-Z]', token):
            filteredTokens.append(token)
    return filteredTokens

#Next step would be to remove the useless tokens. Useless tokens are raw puntuation, numeric tokens, etc.
def stem(filteredTokens) :
    stemList = [stemmer.stem(tok) for tok in filteredTokens]
    return stemList

def tokenizeAndStem(paragraph) :
    tokens = [word for sentence in nltk.sent_tokenize(paragraph) for word in nltk.word_tokenize(sentence)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):  #normal regex search
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [8]:
import sys

reload(sys)
sys.setdefaultencoding('utf8')

tokenizedParagraphList = []
stemmedParagraphList = []
for i in paragraphs :
    tokenizedParagraph = tokenize(i)
    tokenizedParagraphList.extend(tokenizedParagraph)
    stemmedParagraphList.extend(stem(tokenizedParagraph))
print tokenizedParagraphList
print stemmedParagraphList

In [9]:
#create a pandas DataFrame with the stemmed vocabulary as the index and the tokenized words as the column.
#The benefit of this is it provides an efficient way to look up a stem and return a full token. 
#The downside here is that stems to tokens are one to many: the stem 'run' could be associated with 'ran', 'runs', 'running', etc.


vocabFrame = pd.DataFrame({'words': tokenizedParagraphList}, index = stemmedParagraphList)
print 'there are ' + str(vocabFrame.shape[0]) + ' items in vocab_frame'

In [10]:
#for tfidf, there are two important things, max_df & min_df;
#max_df - When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words).
#min_idf - When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature.


from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(stop_words='english', use_idf=True, tokenizer=tokenizeAndStem, ngram_range=(1,3))
#Now fit the vectorizer to synopses
%time tfidf_matrix = tfidfVectorizer.fit_transform(paragraphs)
print tfidf_matrix
print tfidfVectorizer.get_feature_names()

In [11]:
#Now we calculate the cosine similarity as follows. This will gives us the distance which wil help us in clustering in the later stage.

from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)
print dist.shape

In [12]:
thresh=1.05*np.average(dist)
clusters = hcluster.fclusterdata(dist, thresh, criterion="distance")

In [13]:
print(clusters)
numClusters = max(clusters)

In [14]:
clusterStatements = []
clusterIndices = []
for i in range(0,numClusters,1) :
    clusterStatements.append([])
    clusterIndices.append([])
for i in range(0, len(paragraphs), 1) :
    clusterStatements[clusters[i]-1].append(paragraphs[i])
    clusterIndices[clusters[i]-1].append(i)
    #tempClusterStatements[clusters[i]] += paragraphs[i]
print clusterStatements
print clusterIndices

In [15]:
final_sentence_indices=[]
for k in range(0, numClusters, 1) :
    tfidfVectorizerCluster = TfidfVectorizer(stop_words='english', use_idf=True, tokenizer=tokenizeAndStem, ngram_range=(1,3))
    tfidfMatrixCluster = tfidfVectorizerCluster.fit_transform(clusterStatements[k])
    distCluster = 1 - cosine_similarity(tfidfMatrixCluster)
    maxClusterDistance=0
    farthestSentence=0
    for i in range(0, len(distCluster), 1) :
        temp=0
        for j in range(0, len(distCluster[i]), 1) :
            temp+=distCluster[i][j]
        farthestSentence = farthestSentence if maxClusterDistance<temp else i
        maxClusterDistance = maxClusterDistance if maxClusterDistance<temp else temp
    final_sentence_indices.append(clusterIndices[k][farthestSentence])
    print clusterIndices[k][farthestSentence]

print final_sentence_indices

In [16]:
from operator import itemgetter
final_book_sentence_pairs=[]
for i in final_sentence_indices:
    final_book_sentence_pairs.append(book_sentence_pairs[i])
final_book_sentence_pairs=sorted(final_book_sentence_pairs,key=itemgetter(0,1))
#sorted(final_book_sentence_pairs,key=itemgetter(1))
print
print
print final_book_sentence_pairs

In [20]:
import pdfkit

for index, filename in enumerate(glob.glob(filePath + '/*.txt')):
    file = open(filename, 'r')
    txt = file.read()
    txt = txt.replace('\n', '').split('.')
    for i,line in enumerate(final_book_sentence_pairs):
        if(final_book_sentence_pairs[i][0]==index):
            txt[line[1]]='<mark>'+txt[line[1]]+'</mark>'
    txt='.'.join(txt)
    Html_file= open(filename+'.html',"w")
    Html_file.write(txt)
    Html_file.close()
    pdfkit.from_url(filename+'.html', filename+'.pdf')
    #syscall="weasyprint %s %s"%(os.path.splitext(filename)[0]+'.html', os.path.splitext(filename)[0]+'_marked.pdf')
    #weasyprint mypage.html out.pdf
    #print syscall
    #os.system(syscall)